In [1]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

In [2]:
import copy
import gymnasium as gym
import torch
import random

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import Tensor, nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

from pytorch_lightning.callbacks import EarlyStopping

from gymnasium.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

/home/ankdesh/explore/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def display_video(episode=0):
  video_file = open(f'./videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

In [13]:
import torch
import torch.nn as nn

class DQN(nn.Module):
    def __init__(self, hidden_size, obs_size, n_actions):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)    
        )


    def forward(self, x):
        return self.net(x.float())



In [14]:
sample_dqn = DQN(128, 4, 2) 
sample_obs = torch.rand(2, 4) # 2 observations, each with 4 features
chosen_action = sample_dqn(sample_obs) # 2 actions, one for each observation
print (sample_dqn)
print (sample_obs.shape)
print (chosen_action.shape)
print (chosen_action)
_, action = torch.max(chosen_action, dim=1)
torch.tensor([int(x.item()) for x in action])

DQN(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)
torch.Size([2, 4])
torch.Size([2, 2])
tensor([[-0.1568,  0.1362],
        [-0.1180,  0.1327]], grad_fn=<AddmmBackward0>)


tensor([1, 1])

In [15]:
# Define the ε-greedy policy
def epsilon_greedy(state, env, net, epsilon=0.0):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        state = torch.tensor([state]).to(device=device)
        q_values = net(state)
        _, action = torch.max(q_values, dim=1)
        action = torch.tensor([int(x.item()) for x in action])
    return action
    

In [28]:
class ReplaysBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)
    
    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    

In [17]:
from typing import Iterator


class RLDatasest(IterableDataset):

    def __init__(self, buffer, sample_size=200):
        self.buffer = buffer
        self.sample_size = sample_size

    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size):
            yield experience


In [24]:
def create_environment(name):
    env = gym.make(name, render_mode='rgb_array')
    env = RecordVideo(env, video_folder='./videos', episode_trigger=lambda x: x % 50 == 0)
    env = RecordEpisodeStatistics(env)
    return env

In [25]:
env = create_environment("LunarLander-v3")
for episode in range(10):
    done = False
    env.reset()
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, truncated, info = env.step(action)
        

/home/ankdesh/explore/.venv/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/ankdesh/explore/LearnTry-ML/RL/Udemy-Advanced-RL/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [26]:
display_video(8)

In [27]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS


class DeepQLearning(LightningModule):
    def __init__(self, env_name, policy = epsilon_greedy, capacity = 100000, batch_size=256, lr=1e-3, hidden_size=128, 
                 gamma=0.99, loss_fn=F.smooth_l1_loss, optim=AdamW, epsilon_start=1.0, epsilon_end=0.15, epsilon_last_episode=100,
                 samples_per_epoch = 10000, sync_rate=10):
        super(DeepQLearning, self).__init__()
        self.env = create_environment(env_name)
        self.q_net = DQN(hidden_size, self.env.observation_space.shape[0], self.env.action_space.n)

        self.target_q_net = copy.deepcopy(self.q_net)

        self.policy = policy
        self.buffer = ReplaysBuffer(capacity)

        self.save_hyperparameters()

        while len(self.buffer) < self.hparams.samples_per_epoch:
            print(f"Populating buffer: {len(self.buffer)}/{self.hparams.samples_per_epoch * 100}%")
            self.play_episode(epsilon=self.hparams.epsilon_start)

    @torch.no_grad        
    def play_episode(self, policy=None, epsilon=0.0):
        done = False
        state = self.env.reset()
        while not done:
            if policy:
                action = policy(state, self.env, self.q_net, epsilon)
            else:
                action = self.env.action_space.sample() # raondom action to increase exploration
            next_state, reward, done, truncated, info = self.env.step(action)
            experience = (state, action, reward, done, next_state)
            self.buffer.append(experience)
            state = next_state

    def forward(self, x):
        return self.q_net(x)
    
    def configure_optimizers(self):
        q_net_optimizer = self.hparams.optim(self.q_net.parameters(), lr=self.hparams.lr)
        return [q_net_optimizer]
    

    def train_dataloader(self):
        dataset = RLDatasest(self.buffer, self.hparams.samples_per_epoch)
        dataloader = DataLoader(dataset, batch_size=self.hparams.batch_size)
        return dataloader
    
    def training_step(self, batch, batch_idx):
        states, actions, rewards, dones, next_states = batch
        actions = actions.unsqueeze(1)
        rewards = rewards.unsqueeze(1)
        dones = dones.unsqueeze(1)

        state_action_values = self.q_net(states).gather(1, actions)

        next_action_values, _ = self.target_q_net(next_states).max(dim=1, keepdim=True)[0]
        next_action_values[dones] = 0.0

        expected_state_action_values = rewards + self.hparams.gamma * next_action_values 

        loss = self.hparams.loss_fn(state_action_values, expected_state_action_values)

        self.log("episode/Q_error", loss)
        
        return loss
    
    
    def training_epoch_end(self, training_step_outputs):
        epsilon = max(self.hparams.eps_end, 
                      self.hparams.end_start  - self.current_epoch / self.hparams.epsilon_last_episode)
    
